In [1]:
import os
import json
import codecs
import numpy as np
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
params = {
    'batch_size': 64,
    'lr' : 0.001,
    'max_sent_len': 20,
    'epochs': 500,
    'drops' : [0.1]
         }

In [3]:
def extract_data(data_path):
    """
    意图识别抽取出label
    槽位识别与填充作为命名实体识别问题，对每一个字进行实体标注, ate_time', 'B-target', 'I-date_time', 'I-date_time', 'I-operation', 'I-date_time', 'I-date_time']
[ ]:
￼
​B E I O S
    """
    with codecs.open(data_path,"r",encoding="utf-8") as fp:
        data = json.load(fp)
    texts = [example['text'].replace(" ","") for example in data]
    intent_labels = [example['intent'] for example in data]
    
    slots_ners = []
    count = 0
    for example in data:
        if 'entities' in example.keys():
            text = example['text']
            ner = ['O'] * len(text)
            slots = example['entities']
            for key,val in slots.items():
                start_idx = text.find(val)
                end_idx = start_idx + len(val) -1
                if len(val) == 1:
                    ner[start_idx] = 'S-' + key
                else:
                    ner[start_idx] = 'B-' + key
                    ner[end_idx] = 'E-'+ key
                    for idx in range(start_idx+1, end_idx):
                        ner[idx] = 'I-' + key
        else:
            text = example['text']
            ner = ['O'] * len(text)
        slots_ners.append(ner)
    print('texts len: ', len(texts))
    print('intent_lables len: ',len(intent_labels))
    print('slots_ners len: ', len(slots_ners))
    return texts, intent_labels, slots_ners        

In [4]:
# data_path ="./dataset/data_v2.json"
# max_sent_len = params["max_sent_len"]
# texts, intent_labels, slots_ners = extract_data(data_path)pre_ne
# train_text = [d for i , d in enumerate(texts) if i % 10 != 0]
# valid_text = [d for i , d in enumerate(texts) if i % 10 == 0]
# train_intent = [d for i , d in enumerate(intent_labels) if i % 10 != 0]validation_data=valid_dataset
# valid_intent = [d for i , d in enumerate(intent_labels) if i % 10 == 0]
# train_ner = [d for i , d in enumerate(slots_ners) if i % 10 != 0]
# valid_ner = [d for i , d in enumerate(slots_ners) if i % 10 == 0]

In [5]:
data_path ="../dataset/data_v2.json"
max_sent_len = params["max_sent_len"]
texts, intent, slots_ners = extract_data(data_path)
l = len(texts) // params['batch_size']
texts = texts[:l*params['batch_size']]
intent_labels =  intent[:l*params['batch_size']]
slots_ners = slots_ners[:l*params['batch_size']]

train_text = [d for i , d in enumerate(texts) if i % 10 != 0]
train_l = len(train_text) // params['batch_size']
  
valid_text = [d for i , d in enumerate(texts) if i % 10 == 0]
valid_l = len(valid_text) // params['batch_size']
valid_text = valid_text[:valid_l*params['batch_size']]

train_intent = [d for i , d in enumerate(intent_labels) if i % 10 != 0]
train_intent = train_intent[:train_l*params['batch_size']]
valid_intent = [d for i , d in enumerate(intent_labels) if i % 10 == 0]
valid_intent = valid_intent[:valid_l*params['batch_size']]

train_ner = [d for i , d in enumerate(slots_ners) if i % 10 != 0]
train_ner = train_ner[:train_l*params['batch_size']]
valid_ner = [d for i , d in enumerate(slots_ners) if i % 10 == 0]
valid_ner =valid_ner[:valid_l*params['batch_size']]

texts len:  2517
intent_lables len:  2517
slots_ners len:  2517


In [6]:
with open('/home/ai/hgm/Smart_Home/ner_model/char_conv.json', mode='r', encoding='utf-8') as f:
    dicts = json.load(f)
    
char2id = dicts['char2id']
id2char = dicts['id2char']
intent2id = dicts['intent2id']
id2intent = dicts['id2intent']
slot2id = dicts['slot2id']
id2slot = dicts['id2slot']

params['intent_num'] = len(intent2id)
params['slot_num'] = len(slot2id)
params['id2intent'] = id2intent
params['id2slot'] = id2slot


In [7]:
def trans2labelid(vocab, labels, max_sent_len):
    labels = [vocab[label] for label in labels]
    if len(labels) < max_sent_len:
        labels += [0] * (max_sent_len - len(labels))
    else:
        labels = labels[:max_sent_len]
    return labels

In [8]:
def read_data(txt_seqs, intent_labels, slot_ners,char2id,intent2id,slot2id,max_sent_len):
    dataset_text_labels = []
    dataset_intent_labels = []
    dataset_ner_labels = []
    
    for index in range(len(txt_seqs)):
        dataset_text_labels.append(trans2labelid(char2id,txt_seqs[index],max_sent_len))
        dataset_intent_labels.append([intent2id[intent_labels[index]]])
        dataset_ner_labels.append(trans2labelid(slot2id,slot_ners[index],max_sent_len))
    dataset_text_labels = np.array(dataset_text_labels)
    dataset_intent_labels = np.array(dataset_intent_labels)
    dataset_ner_labels = np.array(dataset_ner_labels)
    
    return dataset_text_labels, dataset_intent_labels, dataset_ner_labels 

In [9]:
# tarin_seq, train_intent, train_ner =  read_data(texts, intent_labels, slots_ners,char2id,intent2id,slot2id,max_sent_len) 

In [10]:
tarin_seq, train_intent, train_ner =  read_data(train_text, train_intent, train_ner,char2id,intent2id,slot2id,max_sent_len) 

In [11]:
valid_seq, valid_intent, valid_ner =  read_data(valid_text, valid_intent, valid_ner,char2id,intent2id,slot2id,max_sent_len) 

In [12]:
def Dataset(txt_seqs, dataset_intent_labels, dataset_ner_labels):
    dataset = tf.data.Dataset.from_tensor_slices(({
    "Input" : txt_seqs
    },
    {
        "pre_intent":dataset_intent_labels,
        
        "pre_ner":dataset_ner_labels
    }))
    l = len(txt_seqs)
    dataset = dataset.shuffle(l).batch(params['batch_size'])
    return dataset

In [13]:
train_dataset = Dataset(tarin_seq, train_intent, train_ner)
valid_dataset = Dataset(valid_seq, valid_intent, valid_ner)

In [14]:
params['intent_num'] = len(intent2id)
params['slot_num'] = len(slot2id)
params['id2intent'] = id2intent
params['id2slot'] = id2slot

In [15]:
tf.keras.backend.clear_session()
text_inputs = tf.keras.layers.Input(shape=(20,),name='Input')
embed = tf.keras.layers.Embedding(500,128)(text_inputs)
bilstm = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64,return_sequences=True))(embed)
x_in = tf.keras.layers.LayerNormalization()(bilstm)
x_conv = tf.keras.layers.GlobalAveragePooling1D()(x_in)
pre_intent = tf.keras.layers.Dense(params['intent_num'],\
            activation='sigmoid',name = 'pre_intent')(x_conv)
x_ner  = tf.keras.layers.LayerNormalization()(bilstm)
pre_slot = tf.keras.layers.Dense(params['slot_num'],activation='sigmoid',name = 'pre_ner')(x_ner)
model = tf.keras.Model(text_inputs,[pre_intent,pre_slot])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 20)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 128)      64000       Input[0][0]                      
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 20, 128)      74496       embedding[0][0]                  
__________________________________________________________________________________________________
layer_normalization (LayerNorma (None, 20, 128)      256         bidirectional[0][0]              
_______________________________________________________________________________________

In [16]:
losses = {'pre_intent':'sparse_categorical_crossentropy','pre_ner':'sparse_categorical_crossentropy'}
metrics = { 'pre_intent': ['accuracy'],'pre_ner': ['accuracy']}
optimizer = tf.keras.optimizers.Adam(params['lr'])
model.compile(optimizer, loss=losses, metrics=metrics)

In [17]:
file_path = '../ner_model_weight/model_conv.h5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(file_path,
                                                        save_weights_only=False, save_best_only=True)
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(patience=50, factor=0.5)
callbacks_list = [checkpoint,learning_rate_reduction]

In [18]:
model.fit(train_dataset,epochs=params['epochs'],validation_data=valid_dataset,callbacks=callbacks_list)

Epoch 1/500
35/35 [==============================] - 1s 23ms/step - loss: 5.5688 - pre_intent_loss: 3.4329 - pre_ner_loss: 2.1359 - pre_intent_accuracy: 0.2558 - pre_ner_accuracy: 0.6029 - val_loss: 4.2793 - val_pre_intent_loss: 3.1946 - val_pre_ner_loss: 1.0847 - val_pre_intent_accuracy: 0.2292 - val_pre_ner_accuracy: 0.6151
Epoch 2/500
35/35 [==============================] - 0s 8ms/step - loss: 3.5672 - pre_intent_loss: 2.7606 - pre_ner_loss: 0.8066 - pre_intent_accuracy: 0.2152 - pre_ner_accuracy: 0.7961 - val_loss: 3.1854 - val_pre_intent_loss: 2.5984 - val_pre_ner_loss: 0.5870 - val_pre_intent_accuracy: 0.1771 - val_pre_ner_accuracy: 0.8914
Epoch 3/500
35/35 [==============================] - 0s 7ms/step - loss: 2.5616 - pre_intent_loss: 2.0407 - pre_ner_loss: 0.5208 - pre_intent_accuracy: 0.4826 - pre_ner_accuracy: 0.8977 - val_loss: 2.2502 - val_pre_intent_loss: 1.7797 - val_pre_ner_loss: 0.4705 - val_pre_intent_accuracy: 0.6719 - val_pre_ner_accuracy: 0.9055
Epoch 4/500
35/35 

In [ ]:
# model.fit(tarin_seq, [train_intent, train_ner],epochs=params['epochs'],validation_split=0.1,callbacks=callbacks_list)

Epoch 1/500
72/72 [==============================] - 0s 5ms/step - loss: 0.0210 - pre_intent_loss: 0.0086 - pre_ner_loss: 0.0125 - pre_intent_accuracy: 0.9939 - pre_ner_accuracy: 0.9918 - val_loss: 8.1595 - val_pre_intent_loss: 7.0034 - val_pre_ner_loss: 1.1561 - val_pre_intent_accuracy: 0.3373 - val_pre_ner_accuracy: 0.8533
Epoch 2/500
72/72 [==============================] - 0s 5ms/step - loss: 0.0210 - pre_intent_loss: 0.0086 - pre_ner_loss: 0.0125 - pre_intent_accuracy: 0.9939 - pre_ner_accuracy: 0.9918 - val_loss: 8.1594 - val_pre_intent_loss: 7.0034 - val_pre_ner_loss: 1.1560 - val_pre_intent_accuracy: 0.3373 - val_pre_ner_accuracy: 0.8533
Epoch 3/500
72/72 [==============================] - 0s 5ms/step - loss: 0.0210 - pre_intent_loss: 0.0086 - pre_ner_loss: 0.0125 - pre_intent_accuracy: 0.9939 - pre_ner_accuracy: 0.9918 - val_loss: 8.1596 - val_pre_intent_loss: 7.0035 - val_pre_ner_loss: 1.1561 - val_pre_intent_accuracy: 0.3373 - val_pre_ner_accuracy: 0.8535
Epoch 4/500
72/72 [

In [ ]:
tf.keras.utils.Sequence()

In [ ]:
tf.keras.models.Model.fit_generator()